<a href="https://colab.research.google.com/github/Sihamtahi/DeepL-NoteBooks/blob/main/Convolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow
#

In [2]:

import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.7.0


**Séance du matin**

In [ ]:
import numpy as np

In [ ]:

# Loading data
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

#  Convert the sample data from integers to floating-point numbers
x_train, x_test = x_train / 255, x_test / 255

x_train, x_test = np.expand_dims(x_train, axis= -1) , np.expand_dims(x_test, axis= -1)

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:

#Build a tf.keras.Sequential model by stacking layers 
  tf.keras.layers.Conv2D(32, (4,4), activation='relu'),
  tf.keras.layers.MaxPool2D(),
  tf.keras.layers.LayerNormalization(),
  tf.keras.layers.Conv2D(64, (4,4), activation='relu'),
  tf.keras.layers.MaxPool2D(),
  tf.keras.layers.LayerNormalization(),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.LayerNormalization(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.LayerNormalization(),
  tf.keras.layers.Dense(10, activation='softmax'),
])

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 61s 32ms/step - loss: 0.3151 - accuracy: 0.8989
Epoch 2/5
1875/1875 [==============================] - 60s 32ms/step - loss: 0.1086 - accuracy: 0.9671
Epoch 3/5
1875/1875 [==============================] - 59s 32ms/step - loss: 0.0795 - accuracy: 0.9763
Epoch 4/5
1875/1875 [==============================] - 59s 31ms/step - loss: 0.0653 - accuracy: 0.9808
Epoch 5/5
1875/1875 [==============================] - 59s 31ms/step - loss: 0.0575 - accuracy: 0.9830


In [ ]:
#The Model.evaluate method checks the models performance
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 3s - loss: 0.0604 - accuracy: 0.9834 - 3s/epoch - 10ms/step


[0.06036300212144852, 0.9833999872207642]

**Séance du soir **

In [3]:
MAX_VOCAB= 1000

In [6]:
imdb = tf.keras.datasets.imdb
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=MAX_VOCAB)

17473536/17464789 [==============================] - 0s 0us/step


In [7]:
def preprocess(x, y):
    x = tf.strings.split(x, ' ')
    x = tf.strings.to_number(x, out_type=tf.int32)
    y = tf.cast(y, tf.int32)
    return x, y

In [8]:
def make_dataset(x, y, is_train=False):
    parse = [' '.join([str(tok) for tok in comment]) for comment in x]
    dataset = tf.data.Dataset.from_tensor_slices((parse, y)) #
    dataset = dataset.map(preprocess)
    if is_train:
        dataset = dataset.shuffle(1000)
    dataset = dataset.padded_batch(batch_size=50, padded_shapes=([None], []), padding_values=(0, 0), drop_remainder=True)
    return dataset

In [9]:
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=MAX_VOCAB, output_dim=50, mask_zero=True),
    #tf.keras.layers.LSTM(50, dropout=0.1, recurrent_dropout=0.1, return_sequences=True), #
    tf.keras.layers.LSTM(50, dropout=0., recurrent_dropout=0.),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
train_dataset = make_dataset(x_train, y_train, is_train=True)
test_dataset = make_dataset(x_test, y_test)
model.fit(train_dataset, epochs=3)
model.evaluate(test_dataset)
model.fit(x_train, y_train)
#Collapse

Epoch 1/3
240/500 [=============>................] - ETA: 9:41 - loss: 0.5521 - accuracy: 0.7033